<a href="https://colab.research.google.com/github/Vasugi2003/Big-Data-Analytics/blob/main/Partitioned_Tables_using_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=e52cba926c742f7fd7595fb4f70df887e5b98582f1450adcecb3292990545441
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .appName("PySparkTransformationsExample")\
        .getOrCreate()


In [ ]:
df=spark.read.option("header",True).csv("zipcodes.csv")
df.printSchema()
df.show()

root
 |-- RecordNumber: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- ZipCodeType: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- LocationType: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Long: string (nullable = true)
 |-- Xaxis: string (nullable = true)
 |-- Yaxis: string (nullable = true)
 |-- Zaxis: string (nullable = true)
 |-- WorldRegion: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- LocationText: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Decommisioned: string (nullable = true)
 |-- TaxReturnsFiled: string (nullable = true)
 |-- EstimatedPopulation: string (nullable = true)
 |-- TotalWages: string (nullable = true)
 |-- Notes: string (nullable = true)

+------------+-------+-----------+-------------------+-----+--------------+-----+-------+-----+-----+-----+-----------+-------+--------------------+--------------------+------------

In [ ]:
#partitionBy()
df.write.option("header",True).partitionBy("state")\
  .mode("overwrite").csv("tmp2/zipcodes-state2")
#we have a total of 6 different states hence, it creates 6 directories as shown below. The name of the sub-directory would be the partition column and its value (partition column=value).
#While writing the data as partitions, PySpark eliminates the partition column on the data file and adds partition column & value to the folder name, hence it saves some space on storage.

In [ ]:
#partitionBy() multiple columns
df.write.option("header",True).partitionBy("state","city")\
  .mode("overwrite").csv("tmp2/zipcodes-state-city")

In [ ]:
#Use repartition() and partitionBy() together
df.repartition(2).write.option("header",True).partitionBy("state")\
  .mode("overwrite").csv("tmp2/zipcodes-state-v2")

In [ ]:
#partitionBy() control number of partitions
# Data Skew – Control Number of Records per Partition File
#This is particularly helpful when your data is skewed (Having some partitions with very low records
# and other partitions with high number of records)
df.write.option("header",True) \
        .option("maxRecordsPerFile", 3) \
        .partitionBy("state") \
        .mode("overwrite") \
        .csv("/tmp/zipcodes-state_2")


In [ ]:
dfSinglePart = spark.read.option("header",True)\
  .csv("tmp2/zipcodes-state-city/state=AL/city=SPRINGVILLE")
dfSinglePart.printSchema()
dfSinglePart.show()

root
 |-- RecordNumber: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- ZipCodeType: string (nullable = true)
 |-- LocationType: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Long: string (nullable = true)
 |-- Xaxis: string (nullable = true)
 |-- Yaxis: string (nullable = true)
 |-- Zaxis: string (nullable = true)
 |-- WorldRegion: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- LocationText: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Decommisioned: string (nullable = true)
 |-- TaxReturnsFiled: string (nullable = true)
 |-- EstimatedPopulation: string (nullable = true)
 |-- TotalWages: string (nullable = true)
 |-- Notes: string (nullable = true)

+------------+-------+-----------+------------+-----+------+-----+-----+-----+-----------+-------+---------------+--------------------+-------------+---------------+-------------------+----------+-----+
|RecordNumber|Zipcode|ZipCodeType|LocationType|  La

In [ ]:
parqDF = spark.read.option("header",True) \
                  .csv("/content/tmp2/zipcodes-state-v2/")
parqDF.createOrReplaceTempView("ZIPCODE")
spark\
    .sql("select * from ZIPCODE  where state='AL' and city = 'SPRINGVILLE'") \
    .show()


+------------+-------+-----------+-----------+------------+-----+------+-----+-----+-----+-----------+-------+---------------+--------------------+-------------+---------------+-------------------+----------+-----+-----+
|RecordNumber|Zipcode|ZipCodeType|       City|LocationType|  Lat|  Long|Xaxis|Yaxis|Zaxis|WorldRegion|Country|   LocationText|            Location|Decommisioned|TaxReturnsFiled|EstimatedPopulation|TotalWages|Notes|state|
+------------+-------+-----------+-----------+------------+-----+------+-----+-----+-----+-----------+-------+---------------+--------------------+-------------+---------------+-------------------+----------+-----+-----+
|       54355|  35146|   STANDARD|SPRINGVILLE|     PRIMARY|33.77|-86.47| 0.05|-0.82| 0.55|         NA|     US|Springville, AL|NA-US-AL-SPRINGVILLE|        FALSE|           4046|               7845| 172127599| null|   AL|
+------------+-------+-----------+-----------+------------+-----+------+-----+-----+-----+-----------+-------+------

https://sparkbyexamples.com/pyspark/pyspark-partitionby-example/

 How to Choose a Partition Column When Writing to File system?           
              Let’s assume you have a US census table that contains zip code, city, state, and other columns. Creating a partition on the state, splits the table into around 50 partitions, when searching for a zipcode within a state (state=’CA’ and zipCode =’92704′) results in faster as it needs to scan only in a state=CA partition directory.

Partition on zipcode may not be a good option as you might end up with too many partitions.

Another good example of partition is on the Date column. Ideally, you should partition on Year/Month but not on a date.